<a href="https://colab.research.google.com/github/tedk108/iphs290_cultural_analytics/blob/main/Copy_of_scrape_youtube_transcript_api_20220426.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# YouTube Transcript API

* https://github.com/jdepoix/youtube-transcript-api

# Setup

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

MessageError: ignored

In [ ]:
%cd /gdrive/MyDrive/research/year_2022/colab_scrape/
!ls -altr

/gdrive/MyDrive/research/year_2022/colab_scrape
total 1849
-rw------- 1 root root 165831 Mar 14 14:55  web_selenium_mFool_Navigating_Narratives_20220312.ipynb
-rw------- 1 root root 171406 Mar 14 14:56  web_csc6740_rss_scraper_v1.ipynb
-rw------- 1 root root   3964 Mar 20 05:33  scrape_books_grab_20220318.ipynb
-rw------- 1 root root   8818 Mar 21 21:11  data_cleaning_20220321.ipynb
-rw------- 1 root root 191855 Apr  5 16:46  scrape_sec_edgar-crawler_20220405.ipynb
-rw------- 1 root root 195257 Apr  8 08:26  youtube-video-titles-pca-nlp-with-texthero.ipynb
-rw------- 1 root root  58216 Apr 27 05:43 'scrape_Tesla Earnings Twitter Sentiment Analysis_20220426.ipynb'
-rw------- 1 root root 284367 Apr 28 13:19  scrape_youtube_transcript_api_20220414.ipynb
-rw------- 1 root root 349748 Apr 28 14:26  scrape_youtube_transcript_api_20220426
-rw------- 1 root root 205997 May  7 23:08  scrape_youtube-comments-for-free-no-google-api_20220507.ipynb
-rw------- 1 root root 166900 May 10 04:52  scrape

In [ ]:
!ls -altr

total 1849
-rw------- 1 root root 165831 Mar 14 14:55  web_selenium_mFool_Navigating_Narratives_20220312.ipynb
-rw------- 1 root root 171406 Mar 14 14:56  web_csc6740_rss_scraper_v1.ipynb
-rw------- 1 root root   3964 Mar 20 05:33  scrape_books_grab_20220318.ipynb
-rw------- 1 root root   8818 Mar 21 21:11  data_cleaning_20220321.ipynb
-rw------- 1 root root 191855 Apr  5 16:46  scrape_sec_edgar-crawler_20220405.ipynb
-rw------- 1 root root 195257 Apr  8 08:26  youtube-video-titles-pca-nlp-with-texthero.ipynb
-rw------- 1 root root  58216 Apr 27 05:43 'scrape_Tesla Earnings Twitter Sentiment Analysis_20220426.ipynb'
-rw------- 1 root root 284367 Apr 28 13:19  scrape_youtube_transcript_api_20220414.ipynb
-rw------- 1 root root 349748 Apr 28 14:26  scrape_youtube_transcript_api_20220426
-rw------- 1 root root 205997 May  7 23:08  scrape_youtube-comments-for-free-no-google-api_20220507.ipynb
-rw------- 1 root root 166900 May 10 04:52  scrape_PRAW_Reddit_API_20220509.ipynb
-rw------- 1 roo

# Install and Import Libraries

In [ ]:
!pip install youtube_transcript_api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np

from tqdm._tqdm_notebook import tqdm_notebook
import pandas as pd
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
import seaborn as sns

from youtube_transcript_api import YouTubeTranscriptApi

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
import urllib.parse as urlparse
import re
from pandas.core.common import get_cython_func

# Common Functions


In [ ]:
def url_strip(url_dirty):
  '''
  Give a url string
  Return same url with any enclsoing quotes and/or whiteshapes removed
  '''
  url_clean = url_dirty.strip().strip('\"').strip()
  return url_clean

# Test
# test_url = '"https://www.youtube.com/watch?v=3z2-IyZDAxw&list=PLEv2wfbqEAh_z8XYOgigFnOXXShXxMtmA&index=2"'
# url_strip(test_url)

# Clean up url
# df['url'] = df['url'].apply(lambda x : url_strip(str(x)))
# df.head()


In [ ]:

def get_youtube_id(url):
  '''
  Give a url as a text string
  Extract and return the YouTube ID
  '''
  # Get youtube id
  #http://youtu.be/5Y6HSHwhVlY
  #http://www.youtube.com/embed/5Y6HSHwhVlY?rel=0
  #http://www.youtube.com/watch?v=ZFqlHhCNBOI

  regex = re.compile(r'(https?://)?(www\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/(watch\?v=|embed/|v/|.+\?v=)?(?P<id>[A-Za-z0-9\-=_]{11})')

  match = regex.match(url)

  if not match:
    # print('no match')
    url_return = ''
  else:
    url_return = match.group('id')
  
  return url_return


  # Online Test: https://pythex.org/?regex=(https%3F%3A%2F%2F)%3F(www%5C.)%3F(youtube%7Cyoutu%7Cyoutube-nocookie)%5C.(com%7Cbe)%2F(watch%5C%3Fv%3D%7Cembed%2F%7Cv%2F%7C.%2B%5C%3Fv%3D)%3F(%3FP%3Cid%3E%5BA-Za-z0-9%5C-%3D_%5D%7B11%7D)&test_string=http%3A%2F%2Fyoutu.be%2F5Y6HSHwhVlY%0Ahttp%3A%2F%2Fwww.youtube.com%2Fembed%2F5Y6HSHwhVlY%3Frel%3D0%0Ahttp%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DZFqlHhCNBOI&ignorecase=0&multiline=0&dotall=0&verbose=0

# Test


test_url = 'http://www.youtube.com/embed/5Y6HSHwhVlY?rel=0'
get_youtube_id(test_url)

'5Y6HSHwhVlY'

# Get Transcripts

## Option (a): Download via Bulk URL from Playlist: PyTube

In [ ]:
!pip install pytube

     |████████████████████████████████| 56 kB 5.1 MB/s 


In [ ]:
from pytube import Playlist

video_url_ls = []

playlist = Playlist('https://www.youtube.com/playlist?list=PLCvqKltYUg-L4bKc-F_y-2idxHrARegPY')  # https://www.youtube.com/playlist?list=PL6gx4Cwl9DGCkg2uj3PxUWhMDuTw3VKjM')
print('Number of videos in playlist: %s' % len(playlist.video_urls))
for video_url in playlist.video_urls:
    print(video_url)
    video_url_ls.append(video_url)

# playlist.download_all()

Number of videos in playlist: 717
https://www.youtube.com/watch?v=0fG0kBIA01s
https://www.youtube.com/watch?v=R3rw970q-ZE
https://www.youtube.com/watch?v=dOsfAW-ikng
https://www.youtube.com/watch?v=g5ltmOHcNl4
https://www.youtube.com/watch?v=MnUj2WFZh2Y
https://www.youtube.com/watch?v=peS2Pt756EI
https://www.youtube.com/watch?v=YPFERnpO-QQ
https://www.youtube.com/watch?v=MFffJpUEmNY
https://www.youtube.com/watch?v=F5FlxWpgemQ
https://www.youtube.com/watch?v=Nyn36shgsxM
https://www.youtube.com/watch?v=lgBMYl4SOhA
https://www.youtube.com/watch?v=tS7Cl-2kWX0
https://www.youtube.com/watch?v=0YkIYZfXikc
https://www.youtube.com/watch?v=40paWB4PX2M
https://www.youtube.com/watch?v=irPrP_Fp9sQ
https://www.youtube.com/watch?v=pu5cQyU_Ec0
https://www.youtube.com/watch?v=nW0124_nRF0
https://www.youtube.com/watch?v=6vvE_YGxPhg
https://www.youtube.com/watch?v=KULyVo4i11Y
https://www.youtube.com/watch?v=ni_Ss1ogT90
https://www.youtube.com/watch?v=kt-UpHI7mmE
https://www.youtube.com/watch?v=xWP3WTafDv

## Option (b): Read in CSV file of URLs to download

In [ ]:
# 

video_url_df = pd.DataFrame({'url':video_url_ls})
video_url_df.head()
video_url_df.info()

,url
0,https://www.youtube.com/watch?v=0fG0kBIA01s
1,https://www.youtube.com/watch?v=R3rw970q-ZE
2,https://www.youtube.com/watch?v=dOsfAW-ikng
3,https://www.youtube.com/watch?v=g5ltmOHcNl4
4,https://www.youtube.com/watch?v=MnUj2WFZh2Y


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     717 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [ ]:
video_url_df.to_csv('yt_jomboy_urls.csv')

## Option (c): Read Transcripts Data Files

In [ ]:
!ls

url_scrape_clean.txt  url_scrape.txt


In [ ]:
# BUG: Adding ' around "-quoted strings
# https://github.com/pandas-dev/pandas/issues/41819

url_datafile = 'url_scrape.txt'
url_datafile = 'url_scrape_clean.txt'
url_datafile = 'url_scrape_clean_xls.csv'
url_datafile = 'yt_zacks_research.csv'

df = pd.read_csv(url_datafile) # , sep=',', quotechar='"', header=None) #  quoting=csv.QUOTE_NONE)
# df = pd.read_table('url_scrape.txt', sep=',', header=None, quotechar='"')
# df.columns = ['date', 'title', 'temp_year', 'source', 'url']
df.columns = ['date', 'title', 'source', 'url']
# df['title'] = df['title'] + ", " + df['temp_year']
# df.drop(columns=['temp_year'], inplace=True)


# df['source'] = df['source'].apply(lambda x : x.strip())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    2 non-null      object
 1   title   2 non-null      object
 2   source  2 non-null      object
 3   url     2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [ ]:
print(f"[{df.iloc[1]['source']}]")

[Zacks Investment Research]


In [ ]:
# TODO: Fix/Restore

# df[df.source == ' "Zacks Investment Research" ']

,date,title,source,url


,date,title,source,url
0,03/14/19,"Ahead of Wall Street Monday March 4th, 2019",Zacks Investment Research,https://www.youtube.com/watch?v=qx9YVvQZcjY&li...
1,03/04/19,"Ahead of Wall Street Monday March 4th, 2019",Zacks Investment Research,https://www.youtube.com/watch?v=qx9YVvQZcjY&li...


In [ ]:
# test_url = df.iloc[0]['url'].strip().split('/').strip()
# test_url

In [ ]:
test_url = df.iloc[0]['url'].strip() # .strip('\"').strip()
test_url
get_youtube_id(test_url)

NameError: ignored

In [ ]:
test_url = 'https://www.youtube.com/watch?v=-ZO2M7cUy9I&list=PLVbP054jv0KpV2leJ9HHIMqZEkCPX-iPV&index=6'

get_youtube_id(test_url)

NameError: ignored

In [ ]:
df.iloc[0]['url']

'https://www.youtube.com/watch?v=qx9YVvQZcjY&list=PLEv2wfbqEAh-plrgIj7qeFdUWFw_Y_gGE'

'5Y6HSHwhVlY'

In [ ]:
df.drop(columns=['youttube_id'], inplace=True)

KeyError: ignored

In [ ]:
df.shape

(2, 4)

# Download YouTube Transcripts

In [ ]:
%whos DataFrame

Variable       Type         Data/Info
-------------------------------------
video_url_df   DataFrame                             <...>n\n[717 rows x 1 columns]


In [ ]:
video_url_df.head()
video_url_df.info()

,url
0,https://www.youtube.com/watch?v=0fG0kBIA01s
1,https://www.youtube.com/watch?v=R3rw970q-ZE
2,https://www.youtube.com/watch?v=dOsfAW-ikng
3,https://www.youtube.com/watch?v=g5ltmOHcNl4
4,https://www.youtube.com/watch?v=MnUj2WFZh2Y


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     717 non-null    object
dtypes: object(1)
memory usage: 5.7+ KB


In [ ]:
video_url_df['youtube_id'] = video_url_df['url'].apply(lambda x: get_youtube_id(str(x))) 
video_url_df

,url,youtube_id
0,https://www.youtube.com/watch?v=0fG0kBIA01s,0fG0kBIA01s
1,https://www.youtube.com/watch?v=R3rw970q-ZE,R3rw970q-ZE
2,https://www.youtube.com/watch?v=dOsfAW-ikng,dOsfAW-ikng
3,https://www.youtube.com/watch?v=g5ltmOHcNl4,g5ltmOHcNl4
4,https://www.youtube.com/watch?v=MnUj2WFZh2Y,MnUj2WFZh2Y
...,...,...
712,https://www.youtube.com/watch?v=aI52FQ23SW0,aI52FQ23SW0
713,https://www.youtube.com/watch?v=NP0tk0NvYFw,NP0tk0NvYFw
714,https://www.youtube.com/watch?v=reniQG9mz3o,reniQG9mz3o
715,https://www.youtube.com/watch?v=koRf2ixEsVk,koRf2ixEsVk


In [ ]:
test_str = video_url_df.iloc[1]['url']
print(f'Testing on url: {test_str}')
get_youtube_id(test_str)

Testing on url: https://www.youtube.com/watch?v=R3rw970q-ZE


'R3rw970q-ZE'

In [ ]:
# video_id('"https://www.youtube.com/watch?v=2uek6CPYl_s&list=PLEv2wfbqEAh_z8XYOgigFnOXXShXxMtmA&index=2"')

In [ ]:
"""
df['filename'] = df.apply(lambda x: f'{x.source.split()[0]}_{x.date}', axis=1)
df['filename'] = df['filename'].apply(lambda x: x.replace('/',''))
df.head()
""";

,date,title,source,url,youtube_id,filename
0,03/14/19,"Ahead of Wall Street Monday March 4th, 2019",Zacks Investment Research,https://www.youtube.com/watch?v=qx9YVvQZcjY&li...,qx9YVvQZcjY,Zacks_031419
1,03/04/19,"Ahead of Wall Street Monday March 4th, 2019",Zacks Investment Research,https://www.youtube.com/watch?v=qx9YVvQZcjY&li...,qx9YVvQZcjY,Zacks_030419


In [ ]:
video_id = 'ATF5ju0rpE4'

yt_transcript_ls = YouTubeTranscriptApi.get_transcript(video_id)
print(yt_transcript_ls)

[{'text': '[Music]', 'start': 0.31, 'duration': 10.42}, {'text': "this episode's sponsored by curiosity", 'start': 11.92, 'duration': 3.28}, {'text': 'stream which now comes through with', 'start': 13.84, 'duration': 3.12}, {'text': 'nebula details are in the description', 'start': 15.2, 'duration': 3.28}, {'text': 'box below', 'start': 16.96, 'duration': 3.84}, {'text': 'hello and welcome history buffs my name', 'start': 18.48, 'duration': 4.16}, {'text': "is nick hodges and we're back again with", 'start': 20.8, 'duration': 4.0}, {'text': 'another classic historical epic after', 'start': 22.64, 'duration': 4.32}, {'text': "six years of your requests i've decided", 'start': 24.8, 'duration': 4.239}, {'text': 'that the time is now right for me to do', 'start': 26.96, 'duration': 4.88}, {'text': 'an episode on the last of the mohicans', 'start': 29.039, 'duration': 4.401}, {'text': 'based on the book of the same name by', 'start': 31.84, 'duration': 3.76}, {'text': 'classic american aut

In [ ]:
type(yt_transcript_ls)

list

In [ ]:
import time

In [ ]:
# Get transcripts

transcript_ls = []
transcript_lines_ls = []

url_ls = video_url_df['youtube_id'].to_list()
for i, video_id in enumerate(url_ls):

  print(f'Dictionary #{i}')
  print(f'  YouTube ID: {len(video_id)}')
  # print(f'  adict["text"]: {adict["text"]}\n')
  # print(f'  {adict.items()}')
  yt_transcript_ls = YouTubeTranscriptApi.get_transcript(video_id)
  for j, aline in enumerate(yt_transcript_ls):
    transcript_lines_ls.append(aline['text'])
  transcript_str = ' '.join(transcript_lines_ls)
  transcript_ls.append(transcript_str)
  if i%5 == 0:
    time.sleep(5)

Dictionary #0
  YouTube ID: 11


YouTubeRequestFailed: ignored

In [ ]:
len(transcript_ls)

226

In [ ]:
transcript_ls[:5]

["[Music] this episode's sponsored by curiosity stream which now comes through with nebula details are in the description box below hello and welcome history buffs my name is nick hodges and we're back again with another classic historical epic after six years of your requests i've decided that the time is now right for me to do an episode on the last of the mohicans based on the book of the same name by classic american author james fenimore cooper this movie takes place on the new york frontier in 1757 during the height of the french and indian war where the empires of france and great britain battled for the ultimate prize the north american continent in their titanic struggle both sides used their armies colonists and indigenous peoples to wage war on the other and caught in the middle are our protagonists a british officer duncan hayward safeguarding two sisters alice and cora who are desperately trying to reach their father and safety and the mohican escorts native raised is the 

In [ ]:
video_url_df['transcript'] = transcript_ls

In [ ]:


video_url_df.to_csv()

In [ ]:
datetimestr = 

temp_df = pd.DataFrame({'transcripts':transcript_ls})
temp_df.to_csv(f'yt_transcripts_{datetime_str}')

,transcripts
0,[Music] this episode's sponsored by curiosity ...
1,[Music] this episode's sponsored by curiosity ...
